In [1]:
class BinaryTree:
    def __init__(self, key, leftTree=None, rightTree=None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree

    def setKey(self, key):
        self.key = key

    def getKey(self):
        return self.key

    def getLeftTree(self):
        return self.leftTree

    def getRightTree(self):
        return self.rightTree

    def insertLeft(self, key):
        if self.leftTree is None:
            self.leftTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, leftTree=self.leftTree)
            self.leftTree = new_node

    def insertRight(self, key):
        if self.rightTree is None:
            self.rightTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, rightTree=self.rightTree)
            self.rightTree = new_node

    def printPreorder(self, level=0):
        print(str(level * '-') + str(self.key))
        if self.leftTree is not None:
            self.leftTree.printPreorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPreorder(level + 1)
            
    def printInorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printInorder(level + 1)
        print(str(level * '-') + str(self.key))
        if self.rightTree is not None:
            self.rightTree.printInorder(level + 1)

    def printPostorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printPostorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPostorder(level + 1)
        print(str(level * '-') + str(self.key))


class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.isEmpty():
            return self.items.pop()
        else:
            raise IndexError("Pop from an empty stack")

    def peek(self):
        if not self.isEmpty():
            return self.items[-1]
        else:
            raise IndexError("Peek from an empty stack")

    def size(self):
        return len(self.items)

def tokeniser(exp):
    raw = list(exp.replace(" ", ""))
    tokens = []
    i = 0
    while i < len(raw):
        if raw[i] == '*' and i + 1 < len(raw) and raw[i + 1] == '*':
            tokens.append('**')
            i += 2  # Skip the next character as it's already combined
        else:
            tokens.append(raw[i])
            i += 1

    return tokens


def buildParseTree(exp):
    tokens = tokeniser(exp)
    stack = Stack()
    tree = BinaryTree('?')
    stack.push(tree)

    currentTree = tree

    for t in tokens:
        if t == '(':
            currentTree.insertLeft('?')
            stack.push(currentTree)
            currentTree = currentTree.getLeftTree()
        elif t in ['+', '-', '*', '/', '**']:
            currentTree.setKey(t)
            currentTree.insertRight('?')
            stack.push(currentTree)
            currentTree = currentTree.getRightTree()
        elif t not in ['+', '-', '*', '/', '**', ')']:
            currentTree.setKey(int(t))
            parent = stack.pop()
            currentTree = parent
        elif t == ')':
            currentTree = stack.pop()
        else:
            raise ValueError("Invalid token")

    return tree


def evaluate(tree):
    if tree.getLeftTree() is None and tree.getRightTree() is None:
        return tree.getKey()

    leftTree = tree.getLeftTree()
    rightTree = tree.getRightTree()
    op = tree.getKey()

    if op == '+':
        return evaluate(leftTree) + evaluate(rightTree)
    elif op == '-':
        return evaluate(leftTree) - evaluate(rightTree)
    elif op == '*':
        return evaluate(leftTree) * evaluate(rightTree)
    elif op == '/':
        return evaluate(leftTree) / evaluate(rightTree)
    elif op == '**':
        return evaluate(leftTree) ** evaluate(rightTree)

In [2]:
class VariableCalculator:
    def __init__(self):
        self.variables = {}

    def process_input(self, input_line):
        # Split input into variable and expression
        var, expr = input_line.split('=', 1)
        var = var.strip()
        expr = expr.strip()

        # Evaluate expression
        try:
            # Replace variables in the expression with their values
            for key in self.variables:
                expr = expr.replace(key, str(self.variables[key]))

            # Evaluate the expression safely just in case for rm, -rf DONT REMOVE
            value = eval(expr, {"__builtins__": None}, {})
        except Exception as e:
            # Handle other exceptions
            print(f"{var} = {expr} => None")
            return

        # Store the result in the variables dictionary
        self.variables[var] = value
        print(f"{var} = {expr} => {value}")

# Example usage
calc = VariableCalculator()
inputs = [
    "a = (3 + (4 * 5))",
    "b = a ** 2",
    "c = a + b",
    "d = x"
]

for input_line in inputs:
    calc.process_input(input_line)


a = (3 + (4 * 5)) => 23
b = 23 ** 2 => 529
c = 23 + 529 => 552
d = x => None


In [3]:
exp = '( 2 + ( 4 * 5) )'
tree = buildParseTree(exp)
tree.printPreorder(0)

print (f'The expression: {exp} evaluates to: {evaluate(tree)}')

+
-2
-*
--4
--5
The expression: ( 2 + ( 4 * 5) ) evaluates to: 22


In [4]:
exp = '(3 + (4 * 5))'
tree = buildParseTree(exp)
tree.printPreorder(0)

print (f'The expression: {exp} evaluates to: {evaluate(tree)}')

+
-3
-*
--4
--5
The expression: (3 + (4 * 5)) evaluates to: 23


In [5]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def evaluate_with_variables(tree, variables):
    if tree.getLeftTree() is None and tree.getRightTree() is None:
        key = tree.getKey()
        if is_number(key):
            return float(key)
        return variables.get(key, None)

    leftVal = evaluate_with_variables(tree.getLeftTree(), variables) if tree.getLeftTree() else None
    rightVal = evaluate_with_variables(tree.getRightTree(), variables) if tree.getRightTree() else None
    op = tree.getKey()

    if op == '+':
        return leftVal + rightVal
    elif op == '-':
        return leftVal - rightVal
    elif op == '*':
        return leftVal * rightVal
    elif op == '/':
        return leftVal / rightVal
    elif op == '**':
        return leftVal ** rightVal

def process_assignments(assignments):
    variables = {}
    for var, exp in assignments:
        tree = buildParseTree(exp)
        variables[var] = evaluate_with_variables(tree, variables)
    return variables

# Example Usage
assignments = [
    ('a', '(3 + (4 * 5))'),
    ('b', 'a ** 2'),
    ('c', 'a + b'),
    ('d', 'x')
]

results = process_assignments(assignments)
for var, exp in assignments:
    print(f"{var} = {exp} => {results.get(var, 'None')}")


ValueError: invalid literal for int() with base 10: 'a'

In [ ]:
class VariableCalculator:
    def __init__(self):
        self.variables = {}

    def process_input(self, input_line):
        # Split input into variable and expression
        var, expr = input_line.split('=', 1)
        var = var.strip()
        expr = expr.strip()

        # Evaluate expression
        try:
            # Replace variables in the expression with their values
            for key in self.variables:
                expr = expr.replace(key, str(self.variables[key]))

            # Evaluate the expression safely just in case for rm, -rf DONT REMOVE
            value = eval(expr, {"__builtins__": None}, {})
        except Exception as e:
            # Handle other exceptions
            print(f"{var} = {expr} => None")
            return

        # Store the result in the variables dictionary
        self.variables[var] = value
        print(f"{var} = {expr} => {value}")

# Example usage
calc = VariableCalculator()
inputs = [
    "a = (3 + (4 * 5))",
    "b = a ** 2",
    "c = a + b",
    "d = x"
]

for input_line in inputs:
    calc.process_input(input_line)
